In [1]:
import tweepy as tw

# Más información en: https://docs.tweepy.org

In [2]:
# Crear una cuenta en: https://developer.twitter.com/

# Cambiar estos valores por los tokens personales de cada cuenta
consumer_key= 'abR6wsEZvZBvZsuOyhA3GtSC6'
consumer_secret= 'jYHvbRcB8hkr1gHTflOHwsss9RCIZKyn7Ebonxzw41jYKnG5BZ'
access_token= '1195575657176154112-r6dQgX0MJBWlXnePCY5Sco8GEHPiyy'
access_token_secret= '9Ksf0LPCKWislKo0tWCRSRagnLFj5ToncIL54hTAAflF9'

# Autenticación
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
# API de twitter
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
# Términos de búsqueda y desde cuándo
terminos_busqueda = "#runrun"
fecha = "2021-11-13"
terminos_busqueda += " -filter:retweets"

# Obtener tweets
tweets = tw.Cursor(api.search,
                   q = terminos_busqueda,
                   lang = "es",
                   since = fecha).items(6)
# Iterar y mostrar el texto de los tweets
[tweet.text for tweet in tweets]

['Así es, somos noticia mundial!!! #RunRun https://t.co/RkoynVCJRQ',
 '#RunRun mi zorrito https://t.co/S8VlstjRp3',
 'Solo vengo a decirles que adopten. No sean cómplices del tráfico de animales. \n\nRecuerden lo que le paso al zorrito #RunRun',
 '¿Y para cuando van agarrar a Ron Ron?\nCárcel para los #Corruptos y libertad para #RunRun😢 https://t.co/kypgY0MdDu',
 'Todo listo para mañana. Media Maratón Gandia. #mediamaratongandia #mitjamaratogandia #beplus #vickyfoods #RunRun… https://t.co/bDHQsPJGLz',
 'Hace una semana, llegando a Tehotihuacan, en la carrera El Pescado de Moctezuma  #Runrun #rungdl #Guanajuato… https://t.co/QxxewCZlKO']

In [4]:
# Obtener tweets
tweets = tw.Cursor(api.search,
                   q = terminos_busqueda,
                   lang = "es",
                   since = fecha).items(4)
# Iterar y mostrar algunos metadatos de los tweets
for tweet in tweets:
    print("\nUsuario:", tweet.user.screen_name)
    print("Localizacion:", tweet.user.location)
    print("Fecha y hora:", tweet.created_at)
    print("Tweet:", tweet.text)


Usuario: chipanacatalan
Localizacion: Lima, Peru
Fecha y hora: 2021-11-14 01:59:30
Tweet: Así es, somos noticia mundial!!! #RunRun https://t.co/RkoynVCJRQ

Usuario: rossinabm
Localizacion: 
Fecha y hora: 2021-11-13 22:26:54
Tweet: #RunRun mi zorrito https://t.co/S8VlstjRp3

Usuario: Ribosoma_P
Localizacion: 
Fecha y hora: 2021-11-13 21:56:24
Tweet: Solo vengo a decirles que adopten. No sean cómplices del tráfico de animales. 

Recuerden lo que le paso al zorrito #RunRun

Usuario: gilmerjose_a
Localizacion: Cajamarca, Peru
Fecha y hora: 2021-11-13 21:32:27
Tweet: ¿Y para cuando van agarrar a Ron Ron?
Cárcel para los #Corruptos y libertad para #RunRun😢 https://t.co/kypgY0MdDu


In [5]:
# Conversión a un data frame de Pandas
import pandas as pd

# Obtener tweets
tweets = tw.Cursor(api.search,
                   q = terminos_busqueda,
                   lang = "es",
                   since = fecha).items(4)

# Generar una lista iterable
users_locs = [[tweet.user.screen_name, 
               str(tweet.created_at), 
               tweet.user.location, 
               tweet.text] 
              for tweet in tweets]

# Convertir a un data frame
tweet_text = pd.DataFrame(data = users_locs, 
                          columns = ['usuario', 'fecha', 'lugar', "texto"])
tweet_text

,usuario,fecha,lugar,texto
0,chipanacatalan,2021-11-14 01:59:30,"Lima, Peru","Así es, somos noticia mundial!!! #RunRun https..."
1,rossinabm,2021-11-13 22:26:54,,#RunRun mi zorrito https://t.co/S8VlstjRp3
2,Ribosoma_P,2021-11-13 21:56:24,,Solo vengo a decirles que adopten. No sean cóm...
3,gilmerjose_a,2021-11-13 21:32:27,"Cajamarca, Peru",¿Y para cuando van agarrar a Ron Ron?\nCárcel ...


### Streams

In [9]:
import time 
import json

class Listener(tw.StreamListener):
    contador_tweets = 0    # Contador de tweets

    def __init__(self, tiempo_maximo=60):
        self.tiempo_inicial = time.time()
        self.tiempo_maximo = tiempo_maximo
        self.file = open("stream.txt", "w")
        super(Listener, self).__init__()

    def get_datos(self):
        return self.datos

    def on_data(self, data):
        Listener.contador_tweets += 1
        # Almacenar el tweet en un archivo
        self.file.write(data)
        # Convertir a json para mostrar solo algunos valores
        data = json.loads(data)
        # Mostrar el "texto" del tweet
        print("Tweet " + str(Listener.contador_tweets) + "\n" + 30*"-")
        print(str(data["text"])+"\n")
        if ((time.time() - self.tiempo_inicial) < self.tiempo_maximo) and \
           (Listener.contador_tweets < Listener.max_tweets):
            return True
        else:
            print("Se alcanzó el máximo número de tweets o se llegó al tiempo máximo")
            self.file.close()
            return False

    def getTweetsByHashtag(self, max_tweets, hashtag):
        try:
            Listener.max_tweets = max_tweets
            streaming_api = tw.Stream(auth, Listener(), timeout=60)
            streaming_api.filter(track=[hashtag])
        except KeyboardInterrupt:
            print('Interrupción de teclado')

# Intancia de stream
listener = Listener()
listener.getTweetsByHashtag(4, "peru")

Tweet 1
------------------------------
RT @larepublica_pe: 🚨 #AHORA | 💉 Alcalde de La Victoria alista plan para que solo vacunados con ambas dosis ingresen a Gamarra

Esta medida…

Tweet 2
------------------------------
RT @Gestionpe: Perú acogerá la 52 Asamblea General de la OEA en el 2022, tercera en su historia https://t.co/YIbimx9X4A

Tweet 3
------------------------------
@BelspBreno @UOLNoticias Vc quer um peru? Te arrumo um peruzão!

Tweet 4
------------------------------
RT @juank23_7: @CancilleriaPeru informa sobre el MILLÓN de dosis de #Sinopharm arribadas hoy 13Nov al Perú como parte de la donación del Go…

Se alcanzó el máximo número de tweets o se llegó al tiempo máximo


In [10]:
# Leer el archivo grabado
f = open('stream.txt','r')
datos = []
for i in f:
    dato_formato_json = json.loads(i) 
    print(dato_formato_json)
    datos.append(dato_formato_json)

{'created_at': 'Sun Nov 14 03:48:46 +0000 2021', 'id': 1459729999158816770, 'id_str': '1459729999158816770', 'text': 'RT @larepublica_pe: 🚨 #AHORA | 💉 Alcalde de La Victoria alista plan para que solo vacunados con ambas dosis ingresen a Gamarra\n\nEsta medida…', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 282109564, 'id_str': '282109564', 'name': 'Diegoqg', 'screen_name': 'ciberescritor', 'location': 'El Alto — Perú        Te sigue', 'url': 'http://escritores.foroactivo.com', 'description': 'Escritor, Bohemio. Amigo de mis amigos, monoteísta, enemigo de los fujiapristones !\nQuiero ser uno, del millón de amigos que tu tienes !', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 619, 'friends_count': 620, 'listed_count'

In [11]:
# Convertir el archivo leido en un dataframe de Pandas
df = pd.DataFrame.from_dict(datos, orient='columns')
df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,display_text_range
0,Sun Nov 14 03:48:46 +0000 2021,1459729999158816770,1459729999158816770,RT @larepublica_pe: 🚨 #AHORA | 💉 Alcalde de La...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,"{'id': 282109564, 'id_str': '282109564', 'name...",None,None,None,None,{'created_at': 'Sat Nov 13 14:54:03 +0000 2021...,False,0,0,0,0,"{'hashtags': [{'text': 'AHORA', 'indices': [22...",False,False,low,es,1636861726928,NaN,NaN
1,Sun Nov 14 03:48:47 +0000 2021,1459730001096548353,1459730001096548353,RT @Gestionpe: Perú acogerá la 52 Asamblea Gen...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1307728701908488199, 'id_str': '1307728...",None,None,None,None,{'created_at': 'Sat Nov 13 06:45:01 +0000 2021...,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,es,1636861727390,False,NaN
2,Sun Nov 14 03:48:51 +0000 2021,1459730017064267781,1459730017064267781,@BelspBreno @UOLNoticias Vc quer um peru? Te a...,"<a href=""http://twitter.com/download/android"" ...",False,1.459609e+18,1459608876647600129,2.271482e+09,2271481852,BelspBreno,"{'id': 1436779727692443661, 'id_str': '1436779...",None,None,None,None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,pt,1636861731197,NaN,"[25, 63]"
3,Sun Nov 14 03:48:52 +0000 2021,1459730024475639809,1459730024475639809,RT @juank23_7: @CancilleriaPeru informa sobre ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,"{'id': 1423667706448994305, 'id_str': '1423667...",None,None,None,None,{'created_at': 'Sun Nov 14 03:21:50 +0000 2021...,False,0,0,0,0,"{'hashtags': [{'text': 'Sinopharm', 'indices':...",False,False,low,es,1636861732964,NaN,NaN
